In [1]:
import os
import sys
import urllib.request

# 1. Clone the repository
if not os.path.exists("/kaggle/working/stylegan2-ada-pytorch"):
    !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

# 2. Install Ninja (required for C++ extensions)
!pip install ninja

# 3. Add to python path
sys.path.insert(0, "/kaggle/working/stylegan2-ada-pytorch")

# 4. --- THE CRITICAL FIX ---
# This patches the files that cause the "grid_sampler_2d_backward" error
ops_dir = "/kaggle/working/stylegan2-ada-pytorch/torch_utils/ops"
urls = {
    "grid_sample_gradfix.py": "https://raw.githubusercontent.com/NVlabs/stylegan3/main/torch_utils/ops/grid_sample_gradfix.py",
    "conv2d_gradfix.py": "https://raw.githubusercontent.com/NVlabs/stylegan3/main/torch_utils/ops/conv2d_gradfix.py"
}

print("Applying PyTorch compatibility patches...")
for filename, url in urls.items():
    destination = os.path.join(ops_dir, filename)
    try:
        urllib.request.urlretrieve(url, destination)
        print(f" - Fixed: {filename}")
    except Exception as e:
        print(f" - Error fixing {filename}: {e}")

print("\nSetup Complete. PLEASE RESTART YOUR KERNEL NOW if you ran this for the first time.")

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 8.39 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Applying PyTorch compatibility patches...
 - Fixed: grid_sample_gradfix.py
 - Fixed: conv2d_gradfix.py

Setup Complete. PLEASE RESTART YOUR KERNEL NOW if you ran this for the first time.


In [2]:
# --- CONFIGURATION ---
RAW_DATA_DIR = '/kaggle/input/scratch/anomaly_detection_test_data/good'
DATASET_NAME = 'good_images_64x64.zip'
RESOLUTION = 64 # Use 64 or 128. Do not use 28.
# ---------------------

import os

# Define output path
out_path = f"/kaggle/working/datasets/{DATASET_NAME}"

# Run dataset tool
# --transform=center-crop makes images square
# --width/height resizes them to match model architecture
!python /kaggle/working/stylegan2-ada-pytorch/dataset_tool.py \
    --source {RAW_DATA_DIR} \
    --dest {out_path} \
    --transform=center-crop \
    --width={RESOLUTION} --height={RESOLUTION}

print(f"\nDataset ready at: {out_path}")

  0%|                                                  | 0/4157 [00:00<?, ?it/s]/kaggle/working/stylegan2-ada-pytorch/dataset_tool.py:220: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = PIL.Image.fromarray(img, 'RGB')
/kaggle/working/stylegan2-ada-pytorch/dataset_tool.py:429: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = PIL.Image.fromarray(img, { 1: 'L', 3: 'RGB' }[channels])
100%|██████████████████████████████████████| 4157/4157 [00:27<00:00, 152.40it/s]

Dataset ready at: /kaggle/working/datasets/good_images_64x64.zip


In [3]:
# Create directory for results
!mkdir -p /kaggle/working/training-runs

# Start Training
!python /kaggle/working/stylegan2-ada-pytorch/train.py \
    --outdir=/kaggle/working/training-runs \
    --data=/kaggle/working/datasets/good_images_64x64.zip \
    --gpus=1 \
    --cfg=auto \
    --kimg=1000 \
    --snap=10


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/kaggle/working/datasets/good_images_64x64.zip",
    "use_labels": false,
    "max_size": 4157,
    "xflip": false,
    "resolution": 64
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,


In [4]:
# # 1. UPDATE THIS PATH to your latest .pkl file
# # It will be inside /kaggle/working/training-runs/00000-.../
# model_path = "/kaggle/working/training-runs/00000-good_images_64x64-auto1/network-snapshot-000200.pkl"

# # 2. Generate
# # --trunc=0.7 is the "Quality Filter". 
# # Lower (0.5) = Very realistic, less diverse. 
# # Higher (1.0) = More diverse, potential artifacts.
# !python /kaggle/working/stylegan2-ada-pytorch/generate.py \
#     --outdir=/kaggle/working/final_output \
#     --trunc=0.7 \
#     --seeds=0-499 \
#     --network={model_path}

# print("Images generated in /kaggle/working/final_output")

In [5]:
# import os
# import urllib.request

# # Define the target directory based on your traceback path
# # Your error came from: /kaggle/working/stylegan2-ada-pytorch/stylegan2-ada-pytorch/...
# ops_dir = "/kaggle/working/stylegan2-ada-pytorch/stylegan2-ada-pytorch/torch_utils/ops"

# # URLs for the fixed files from the official StyleGAN3 repo
# urls = {
#     "grid_sample_gradfix.py": "https://raw.githubusercontent.com/NVlabs/stylegan3/main/torch_utils/ops/grid_sample_gradfix.py",
#     "conv2d_gradfix.py": "https://raw.githubusercontent.com/NVlabs/stylegan3/main/torch_utils/ops/conv2d_gradfix.py"
# }

# if os.path.exists(ops_dir):
#     print(f"Patching files in {ops_dir}...")
#     for filename, url in urls.items():
#         destination = os.path.join(ops_dir, filename)
#         try:
#             urllib.request.urlretrieve(url, destination)
#             print(f"Successfully updated {filename}")
#         except Exception as e:
#             print(f"Error updating {filename}: {e}")
# else:
#     print(f"Error: Directory {ops_dir} does not exist. Check your repository structure.")